In [1]:
import pandas as pd
import os
from pathlib import Path
import sys

# Read the deals CSV using absolute paths
df = pd.read_csv('/Users/elliottoates/Desktop/streamlit-image-review/stonegate/deals.csv')
dictionary = df.to_dict(orient='records')

# Get the list of image folders using absolute paths
images_dir = Path('/Users/elliottoates/Desktop/streamlit-image-review/stonegate/Images')
image_folders = [folder.name for folder in images_dir.iterdir() if folder.is_dir()]

# Function to normalize text for matching (remove spaces, underscores, and convert to lowercase)
def normalize_text(text):
    return text.strip().lower().replace(' ', '').replace('_', '')

# Function to find matching image folder for a pub_name
def find_image_folder(pub_name, image_folders):
    """
    Find the best matching image folder for a pub_name.
    Returns the folder path if found, None otherwise.
    """
    # Normalize pub_name
    normalized_pub_name = normalize_text(pub_name)
    
    # Create a mapping of normalized folder names to actual folder paths
    folder_mapping = {}
    for folder in image_folders:
        normalized_folder = normalize_text(folder)
        folder_mapping[normalized_folder] = str(images_dir / folder)
    
    # Direct match with normalized names
    if normalized_pub_name in folder_mapping:
        return folder_mapping[normalized_pub_name]
    
    # Partial match (if pub_name contains folder name or vice versa)
    for normalized_folder, folder_path in folder_mapping.items():
        if (normalized_pub_name in normalized_folder or 
            normalized_folder in normalized_pub_name):
            return folder_path
    
    return None

# Function to get image priority score for sorting
def get_image_priority(image_path):
    """
    Get priority score for image sorting.
    Lower score = higher priority.
    """
    filename = os.path.basename(image_path).lower()
    
    # Priority 0: Ends with "main" (highest priority)
    if filename.endswith('main'):
        return 0
    
    # Priority 1: Contains "main" but doesn't end with it
    if 'main' in filename and not filename.endswith('main'):
        return 1
    
    # Priority 2: All other images (will be sorted by quality later)
    return 2

# Function to estimate image quality based on file size
def get_image_quality_score(image_path):
    """
    Estimate image quality based on file size.
    Larger files generally mean higher quality.
    """
    try:
        file_size = os.path.getsize(image_path)
        return file_size
    except:
        return 0

# Function to get image files from a folder with relative paths and proper sorting
def get_images_from_folder(folder_path):
    """
    Get all image files from a folder, sorted by priority and quality.
    Returns a list of relative image file paths.
    """
    if not folder_path or not os.path.exists(folder_path):
        return []
    
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'}
    image_files = []
    
    try:
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if os.path.isfile(file_path):
                file_ext = os.path.splitext(file)[1].lower()
                if file_ext in image_extensions:
                    # Create relative path: Images/folder_name/filename
                    folder_name = os.path.basename(folder_path)
                    relative_path = f"Images/{folder_name}/{file}"
                    image_files.append(relative_path)
    except Exception as e:
        print(f"Error reading folder {folder_path}: {e}")
    
    # Sort images by priority and quality
    def sort_key(image_path):
        priority = get_image_priority(image_path)
        # Get the full path for quality scoring
        full_image_path = os.path.join(folder_path, os.path.basename(image_path))
        quality = get_image_quality_score(full_image_path)
        return (priority, -quality)  # Negative quality so larger files come first
    
    return sorted(image_files, key=sort_key)

# Add image folder paths and image files to each deal in the dictionary
for deal in dictionary:
    deal['image_folder_path'] = find_image_folder(deal['pub_name'], image_folders)
    deal['image_files'] = get_images_from_folder(deal['image_folder_path'])

# Get all deals without images
deals_without_images = [deal for deal in dictionary if not deal['image_files']]

# Print count of deals without images
print(f"Total deals without images: {len(deals_without_images)}")

# Print distinct pub names and count of pubs without images
distinct_pub_names_without_images = sorted(set(deal['pub_name'] for deal in deals_without_images))
print(f"Distinct pub names without images ({len(distinct_pub_names_without_images)} total):")
print("=" * 50)

# Print just the distinct pub names
for pub_name in distinct_pub_names_without_images:
    print(pub_name)

# Show example of sorted images for a deal with images
print("\n" + "=" * 50)
print("Example of sorted images for a deal with images:")
for deal in dictionary:
    if deal['image_files']:
        print(f"\nDeal: {deal['pub_name']}")
        print(f"Images ({len(deal['image_files'])} total):")
        for i, img in enumerate(deal['image_files'][:5]):  # Show first 5
            priority = get_image_priority(img)
            print(f"  Position {i}: {os.path.basename(img)} (Priority: {priority})")
        break

Total deals without images: 36
Distinct pub names without images (18 total):
Gassys Cardiff
Prince Of Teck Earls Court
Tank & Paddle Manchester Printworks
Temperance Fulham
The Bay & Bracket Victoria
The Block & Gasket Burgess Hill
The Block & Gasket Sale
The Boundary Reading
The Bridge Tap London
The Cannick Tapps London
The Centurion Colchester
The Cider Press Bristol
The Distillery Leicester
The Dockyard Portsmouth
The Faraday Epsom
The Garratt And Gauge Wimbledon
The Green Dragon Croydon
The Joiners London

Example of sorted images for a deal with images:


In [6]:
deal_with_images = [deal for deal in dictionary if deal['image_files']]

for deal in deal_with_images:
    print(deal)
    break

In [7]:
import pandas as pd
import boto3
import requests
from dotenv import load_dotenv
import os
import oracledb
import tempfile
from pathlib import Path
from PIL import Image
import shutil

# Load environment variables
load_dotenv()

# Configuration settings
AWS_CONFIG = {
    'aws_access_key_id': os.getenv('AWS_ACCESS_KEY_ID'),
    'aws_secret_access_key': os.getenv('AWS_SECRET_ACCESS_KEY')
}

S3_CONFIG = {
    'bucket_name': os.getenv('S3_BUCKET_NAME', 'static.wowcher.co.uk')
}

# Oracle configuration
ORACLE_CONFIG = {
    'user': os.getenv("ORACLE_USER"),
    'password': os.getenv("ORACLE_PASSWORD"),
    'dsn': os.getenv("ORACLE_DSN")
}

# Target sizes for image variants
TARGET_SIZES = {
    "": (777, 520),
    "-cashback-promo": (126, 90),
    "-email": (640, 428),
    "-thumb": (105, 70),
    "-promo": (172, 115),
    "-user": (50, 50),
    "-deal-bonus": (278, 182),
    "-iphone-medium": (151, 106),
    "-iphone-small": (80, 53),
    "-iphone-promo": (640, 428),
    "-iphone-thumb": (210, 140),
    "-travel-main": (777, 520),
    "-2-per-row": (470, 315),
    "-3-per-row": (310, 210),
}

def uncrop_image(image, target_aspect_ratio):
    """
    Adjust image to target aspect ratio by adding padding if needed
    """
    original_width, original_height = image.size
    original_aspect = original_width / original_height
    
    if abs(original_aspect - target_aspect_ratio) < 1e-6:
        return image
    
    if original_aspect > target_aspect_ratio:
        # Image is too wide, add padding top/bottom
        new_height = int(original_width / target_aspect_ratio)
        new_image = Image.new('RGB', (original_width, new_height), (255, 255, 255))
        paste_y = (new_height - original_height) // 2
        new_image.paste(image, (0, paste_y))
    else:
        # Image is too tall, add padding left/right
        new_width = int(original_height * target_aspect_ratio)
        new_image = Image.new('RGB', (new_width, original_height), (255, 255, 255))
        paste_x = (new_width - original_width) // 2
        new_image.paste(image, (paste_x, 0))
    
    return new_image

def get_file_extension_and_format(image_path):
    """
    Get the file extension and format for an image file.
    Returns (extension, format, content_type)
    """
    # Get the actual file extension
    file_ext = os.path.splitext(image_path)[1].lower()
    
    # Map extensions to formats and content types
    extension_map = {
        '.jpg': ('jpg', 'JPEG', 'image/jpeg'),
        '.jpeg': ('jpg', 'JPEG', 'image/jpeg'),
        '.png': ('png', 'PNG', 'image/png'),
        '.gif': ('gif', 'GIF', 'image/gif'),
        '.bmp': ('bmp', 'BMP', 'image/bmp'),
        '.tiff': ('tiff', 'TIFF', 'image/tiff'),
        '.webp': ('webp', 'WEBP', 'image/webp')
    }
    
    if file_ext in extension_map:
        return extension_map[file_ext]
    else:
        # Default to jpg if unknown format
        return ('jpg', 'JPEG', 'image/jpeg')

def generate_variants(image_id, original_image, temp_dir):
    """
    Generate all required image variants and return their file paths
    Handles different input formats but always outputs JPG for consistency
    """
    variant_files = {}
    
    # Generate main variant first
    first_variant_size = TARGET_SIZES[""]
    first_variant_aspect = first_variant_size[0] / first_variant_size[1]
    base_variant = uncrop_image(original_image, first_variant_aspect)
    base_variant = base_variant.resize(first_variant_size, Image.LANCZOS)
    
    # Save main variant (always as JPG for consistency)
    main_path = os.path.join(temp_dir, f"{image_id}.jpg")
    base_variant.save(main_path, "JPEG")
    variant_files[""] = main_path
    
    # Generate all other variants
    for suffix, (w, h) in TARGET_SIZES.items():
        if suffix == "":
            continue
            
        filename = f"{image_id}{suffix}.jpg"
        variant_path = os.path.join(temp_dir, filename)
        
        if suffix == "-user":
            # Special handling for user avatar (square)
            orig_aspect = original_image.width / original_image.height
            if abs(orig_aspect - 1.0) < 1e-6:
                resized = original_image.resize((w, h), Image.LANCZOS)
            else:
                square_variant = uncrop_image(original_image, 1.0)
                resized = square_variant.resize((w, h), Image.LANCZOS)
        else:
            resized = base_variant.resize((w, h), Image.LANCZOS)
        
        resized.save(variant_path, "JPEG")
        variant_files[suffix] = variant_path
    
    return variant_files

def get_new_oracle_image_id():
    """
    Get a new image ID from Oracle sequence
    """
    connection = oracledb.connect(**ORACLE_CONFIG)
    cursor = connection.cursor()
    
    try:
        cursor.execute("SELECT deal_voucher_image_seq.NEXTVAL FROM dual")
    except Exception as e:
        try:
            cursor.execute("SELECT product_image_seq.NEXTVAL FROM dual")
        except Exception as e2:
            cursor.close()
            connection.close()
            raise Exception(f"Could not find a suitable sequence for images. Tried deal_voucher_image_seq: {e}, product_image_seq: {e2}")
    
    new_image_id = cursor.fetchone()[0]
    connection.commit()
    cursor.close()
    connection.close()
    return new_image_id

def upload_variants_to_s3(variant_files, deal_id, new_image_id):
    """
    Upload all image variants to S3 with the new image ID
    """
    try:
        s3_client = boto3.client('s3', **AWS_CONFIG)
        bucket_name = S3_CONFIG['bucket_name']
        uploaded_urls = {}
        
        for suffix, local_file_path in variant_files.items():
            # Create new S3 key with new image ID and suffix
            new_key = f"images/deal/{deal_id}/{new_image_id}{suffix}.jpg"
            
            # Upload the file
            with open(local_file_path, 'rb') as file_data:
                s3_client.upload_fileobj(
                    file_data,
                    bucket_name,
                    new_key,
                    ExtraArgs={
                        'ContentType': 'image/jpeg',
                        'CacheControl': 'no-cache'
                    }
                )
            
            final_url = f"https://{bucket_name}/{new_key}"
            uploaded_urls[suffix] = final_url
            print(f"✅ Uploaded variant: {new_image_id}{suffix}.jpg")
        
        return uploaded_urls
        
    except Exception as e:
        print(f"Error uploading variants to S3: {str(e)}")
        raise

def get_current_image_positions(deal_id):
    """
    Get all current images for a deal with their positions from Oracle
    """
    connection = oracledb.connect(**ORACLE_CONFIG)
    cursor = connection.cursor()
    
    cursor.execute("""
        SELECT ID, POSITION, FILE_NAME, RESOURCE_PATH, CAPTION, ALT_TAG, EXTENSION, HAS_IPHONE_IMG
        FROM DEAL_VOUCHER_IMAGE 
        WHERE DEAL_VOUCHER_ID = :deal_id 
        AND STATUS_ID = 1
        ORDER BY POSITION
    """, {"deal_id": deal_id})
    
    results = cursor.fetchall()
    cursor.close()
    connection.close()
    
    return results

def calculate_alternating_positions(existing_images, new_images_count):
    """
    Calculate positions for new images alternating with existing ones.
    First new image goes to position 0, then alternate.
    """
    existing_positions = [img[1] for img in existing_images]  # Get existing positions
    
    # If no existing images, just use sequential positions starting from 0
    if not existing_images:
        return list(range(new_images_count))
    
    # Calculate positions for new images
    new_positions = []
    for i in range(new_images_count):
        if i == 0:
            # First new image goes to position 0
            new_positions.append(0)
        else:
            # Alternate: new image position = i * 2
            new_positions.append(i * 2)
    
    # If we have more new images than existing, continue sequentially after alternating
    if new_images_count > len(existing_images):
        # After alternating, continue with sequential positions
        start_seq = len(existing_images) * 2
        for i in range(len(existing_images), new_images_count):
            new_positions[i] = start_seq + (i - len(existing_images))
    
    return new_positions

def update_existing_image_positions(deal_id, new_positions):
    """
    Update positions of existing images to make room for new alternating images.
    Move existing images to odd positions (1, 3, 5, etc.)
    """
    connection = oracledb.connect(**ORACLE_CONFIG)
    cursor = connection.cursor()
    
    try:
        # Get current images
        current_images = get_current_image_positions(deal_id)
        
        if not current_images:
            return True  # No existing images to update
        
        # Move existing images to new positions (odd positions: 1, 3, 5, etc.)
        for i, img in enumerate(current_images):
            new_position = (i * 2) + 1  # 1, 3, 5, 7, etc.
            
            cursor.execute("""
                UPDATE DEAL_VOUCHER_IMAGE 
                SET POSITION = :new_position
                WHERE ID = :image_id AND DEAL_VOUCHER_ID = :deal_id
            """, {
                "new_position": new_position,
                "image_id": img[0],
                "deal_id": deal_id
            })
        
        connection.commit()
        return True
        
    except Exception as e:
        print(f"Error updating existing image positions: {str(e)}")
        if connection:
            connection.rollback()
        return False
        
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

def insert_image_to_oracle(new_image_id, deal_id, file_name, position=0):
    """
    Insert new image record into Oracle DEAL_VOUCHER_IMAGE table with proper format handling
    """
    connection = None
    cursor = None
    
    try:
        connection = oracledb.connect(**ORACLE_CONFIG)
        cursor = connection.cursor()
        
        # Use full S3 URL for resource_path (variants are always .jpg but keep original extension in DB)
        resource_path = f"https://static.wowcher.co.uk/images/deal/{deal_id}"
        
        cursor.execute("""
            INSERT INTO DEAL_VOUCHER_IMAGE (
                ID, DEAL_VOUCHER_ID, RESOURCE_PATH, STATUS_ID,
                FILE_NAME, CAPTION, POSITION, ALT_TAG,
                EXTENSION, HAS_IPHONE_IMG, CREATED_BY_USER_ID, CREATED_DATE
            ) VALUES (
                :new_image_id, :deal_id, :resource_path, 1,
                :file_name, :caption, :position, :alt_tag,
                :extension, :has_iphone_img, 18282217, CURRENT_TIMESTAMP
            )
        """, {
            "new_image_id": new_image_id,
            "deal_id": deal_id,
            "resource_path": resource_path,
            "file_name": file_name,
            "caption": f"Stonegate image {new_image_id}",
            "position": position,
            "alt_tag": f"Stonegate",
            "extension": "jpg",  # Always jpg to match S3 files
            "has_iphone_img": 0
        })
        
        connection.commit()
        return True
        
    except Exception as e:
        print(f"Error inserting image to Oracle: {str(e)}")
        if connection:
            connection.rollback()
        return False
        
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

def process_stonegate_image(deal, image_path, position):
    """
    Process a single Stonegate image with proper format handling and alternating position logic
    """
    temp_dir = None
    
    try:
        # Create temporary directory
        temp_dir = tempfile.mkdtemp()
        
        # Get file extension and format info
        original_ext, original_format, content_type = get_file_extension_and_format(image_path)
        
        print(f"Processing image: {os.path.basename(image_path)} (Format: {original_format})")
        
        # Load the image
        original_image = Image.open(image_path)
        
        # Ensure RGB mode for processing
        if original_image.mode != "RGB":
            original_image = original_image.convert("RGB")
        
        # Get new Oracle image ID
        new_image_id = get_new_oracle_image_id()
        
        # Generate all variants (always output as JPG)
        print(f"Generating variants for image {new_image_id}...")
        variant_files = generate_variants(new_image_id, original_image, temp_dir)
        
        # Upload all variants to S3
        uploaded_urls = upload_variants_to_s3(variant_files, deal['deal_id'], new_image_id)
        
        # Insert into Oracle with the calculated position
        file_name = f"{new_image_id}.jpg"
        oracle_success = insert_image_to_oracle(new_image_id, deal['deal_id'], file_name, position)
        
        if oracle_success:
            return {
                'success': True,
                'new_image_id': new_image_id,
                'deal_id': deal['deal_id'],
                'pub_name': deal['pub_name'],
                'uploaded_urls': uploaded_urls,
                'variants_count': len(variant_files),
                'position': position,
                'original_format': original_format,
                'processed_format': 'jpg'
            }
        else:
            return {
                'success': False,
                'error': 'Failed to insert into Oracle'
            }
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        return {
            'success': False,
            'error': str(e)
        }
        
    finally:
        # Clean up temporary directory
        if temp_dir and os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# Test function for specific deal with complete alternating logic
def test_specific_deal(dictionary, target_deal_id):
    """
    Test the alternating process on a specific deal with proper position handling
    """
    # Find the specific deal
    target_deal = None
    for deal in dictionary:
        if deal['deal_id'] == target_deal_id:
            target_deal = deal
            break
    
    if not target_deal:
        print(f"Deal ID {target_deal_id} not found!")
        return
    
    if not target_deal['image_files']:
        print(f"Deal {target_deal_id} ({target_deal['pub_name']}) has no images!")
        return
    
    print(f"Testing with deal: {target_deal['pub_name']}")
    print(f"Deal ID: {target_deal['deal_id']}")
    print(f"Local images: {len(target_deal['image_files'])} files")
    
    # Check file formats
    for i, image_file in enumerate(target_deal['image_files'][:3]):  # Check first 3
        image_path = os.path.join('/Users/elliottoates/Desktop/streamlit-image-review/stonegate', image_file)
        ext, fmt, content_type = get_file_extension_and_format(image_path)
        print(f"  Image {i+1}: {os.path.basename(image_file)} (Format: {fmt})")
    
    # Get existing images from Oracle
    existing_images = get_current_image_positions(target_deal['deal_id'])
    print(f"Existing images in Oracle: {len(existing_images)}")
    
    if existing_images:
        print("Existing images in database:")
        for img in existing_images:
            print(f"  ID: {img[0]}, Position: {img[1]}, File: {img[2]}")
    
    # Calculate positions for new images
    new_positions = calculate_alternating_positions(existing_images, len(target_deal['image_files']))
    print(f"\nNew image positions: {new_positions}")
    
    # Show the alternating pattern
    print("\nAlternating pattern:")
    all_positions = []
    for i, pos in enumerate(new_positions):
        all_positions.append(f"New{i+1}(pos {pos})")
        if i < len(existing_images):
            all_positions.append(f"Existing{i+1}(pos {existing_images[i][1] + 1})")
    
    print(" → ".join(all_positions))
    
    # Update existing image positions first
    print(f"\nUpdating existing image positions...")
    update_success = update_existing_image_positions(target_deal['deal_id'], new_positions)
    
    if not update_success:
        print("❌ Failed to update existing image positions!")
        return
    
    print("✅ Successfully updated existing image positions")
    
    # Process just the first image for testing
    if target_deal['image_files']:
        image_path = os.path.join('/Users/elliottoates/Desktop/streamlit-image-review/stonegate', target_deal['image_files'][0])
        print(f"\nProcessing first image: {target_deal['image_files'][0]} at position {new_positions[0]}")
        
        result = process_stonegate_image(target_deal, image_path, new_positions[0])
        
        if result['success']:
            print(f"✅ SUCCESS!")
            print(f"   New Image ID: {result['new_image_id']}")
            print(f"   Position: {result['position']}")
            print(f"   Variants created: {result['variants_count']}")
            print(f"   Original format: {result['original_format']}")
            print(f"   Processed as: {result['processed_format']}")
        else:
            print(f"❌ FAILED: {result['error']}")
    
    return result

# Usage:

In [ ]:
# CLEAN VERSION - Process all deals without progress bar
def process_all_deals_clean(dictionary, max_deals=None):
    """
    Process ALL deals that have local images with minimal output
    """
    
    # Filter deals that have images
    deals_with_images = [deal for deal in dictionary if deal['image_files']]
    
    if not deals_with_images:
        print("❌ No deals with images found!")
        return
    
    # Limit processing if max_deals is specified
    if max_deals:
        deals_with_images = deals_with_images[:max_deals]
    
    print(f"🚀 Processing {len(deals_with_images)} deals with images...")
    
    # Track overall statistics
    total_deals = len(deals_with_images)
    successful_deals = 0
    failed_deals = 0
    total_images_processed = 0
    total_variants_created = 0
    
    deal_results = []
    
    # Process each deal
    for deal_idx, deal in enumerate(deals_with_images, 1):
        try:
            # Get existing images and calculate positions
            existing_images = get_current_image_positions(deal['deal_id'])
            new_positions = calculate_alternating_positions(existing_images, len(deal['image_files']))
            
            # Update existing image positions if needed
            if existing_images:
                update_success = update_existing_image_positions(deal['deal_id'], new_positions)
                if not update_success:
                    failed_deals += 1
                    deal_results.append({
                        'deal_id': deal['deal_id'],
                        'pub_name': deal['pub_name'],
                        'success': False,
                        'error': 'Failed to update existing positions'
                    })
                    continue
            
            # Process all images for this deal (silently)
            deal_successful_images = 0
            deal_failed_images = 0
            
            for i, image_file in enumerate(deal['image_files']):
                image_path = os.path.join('/Users/elliottoates/Desktop/streamlit-image-review/stonegate', image_file)
                position = new_positions[i]
                
                # Temporarily suppress prints from process_stonegate_image
                old_stdout = sys.stdout
                sys.stdout = open('/dev/null', 'w')
                
                try:
                    result = process_stonegate_image(deal, image_path, position)
                    
                    if result['success']:
                        deal_successful_images += 1
                        total_variants_created += result['variants_count']
                    else:
                        deal_failed_images += 1
                finally:
                    sys.stdout.close()
                    sys.stdout = old_stdout
            
            # Track results
            total_images_processed += deal_successful_images
            
            if deal_failed_images == 0:
                successful_deals += 1
                deal_results.append({
                    'deal_id': deal['deal_id'],
                    'pub_name': deal['pub_name'],
                    'success': True,
                    'images_processed': deal_successful_images,
                    'total_images': len(deal['image_files'])
                })
            else:
                failed_deals += 1
                deal_results.append({
                    'deal_id': deal['deal_id'],
                    'pub_name': deal['pub_name'],
                    'success': False,
                    'images_processed': deal_successful_images,
                    'total_images': len(deal['image_files']),
                    'failed_images': deal_failed_images
                })
        
        except Exception as e:
            failed_deals += 1
            deal_results.append({
                'deal_id': deal['deal_id'],
                'pub_name': deal['pub_name'],
                'success': False,
                'error': str(e)
            })
    
    # Show final summary
    print("=" * 60)
    print(" PROCESSING COMPLETE")
    print("=" * 60)
    print(f" Deals processed: {total_deals}")
    print(f" Successful: {successful_deals}")
    print(f" Failed: {failed_deals}")
    print(f"  Images processed: {total_images_processed}")
    print(f" Variants created: {total_variants_created}")
    
    # Only show failed deals if any
    if failed_deals > 0:
        print(f"\n FAILED DEALS:")
        for result in deal_results:
            if not result['success']:
                error_msg = result.get('error', f"{result.get('failed_images', 0)} images failed")
                print(f"   • {result['pub_name']} (ID: {result['deal_id']}) - {error_msg}")
    
    return deal_results


In [ ]:
process_all_deals_clean(dictionary)

In [ ]:
# Print all deal_ids in dictionary if images are not empty
deal_ids_with_images = [str(deal['deal_id']) for deal in dictionary if deal.get('image_files')]
print(",".join(deal_ids_with_images))
